In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os

out_folder = 'outputs_bme280'
out_path = os.path.join(os.getcwd(), out_folder)
if not os.path.exists(out_path):
    os.makedirs(out_path)
    
data = pd.read_csv("data_2.csv", index_col=None)
data['date'] = pd.to_datetime(data['date'])

'''
- API IMGW https://danepubliczne.imgw.pl/apiinfo
- data for all stations: https://danepubliczne.imgw.pl/api/data/synop/station
    - MLAWA
    - OLSZTYN
    - TORUN
- FREE API:
    https://open-meteo.com/
    - THIS IS FOR ELGNOWO:   https://api.open-meteo.com/v1/forecast?latitude=53.45&longitude=19.95&current=temperature_2m
'''
df = data.copy()
print(df.shape)

out_miute = pd.DataFrame(columns = ['date','temp', 'humidity', 'pressure', 'altitude'])
out_hour = pd.DataFrame(columns = ['date', 'hour', 'temp', 'humidity', 'pressure', 'altitude'])

minute = df['date'][0].minute # datetime.now().minute
hour = df['date'][0].hour # datetime.now().hour
date = df['date'][0].date() #datetime.now().date()


for ind in df.index:
    # if a miute changes then the data is saved in csv
    if df['date'][ind].minute > minute:
        date = df['date'][ind]
        temp = df['temp'][ind]
        humidity = df['humidity'][ind]
        pressure = df['pressure'][ind]
        altitude = df['altitude'][ind]
        # adding data to the dataframe
        out_miute.loc[len(out_miute.index)] = [date, temp, humidity, pressure, altitude]
        # save to csv ??....
    

    # if the hour changes then the mean from previouse hour is saved
    if df['date'][ind].hour > hour:
        # select readings from previouse hour
        end_time = df['date'][ind] # datetime.no()
        start_time = end_time - timedelta(hours=1)
        selected_rows = out_miute[(out_miute['date'] >= start_time) & (out_miute['date'] <= end_time)]
        mean = selected_rows.mean()
        # saves means to out_hour
        out_hour.loc[len(out_hour.index)] = [df['date'][ind], df['date'][ind].hour, mean['temp'], mean['humidity'], mean['pressure'], mean['altitude']]
        # save to csv ??....
    
    
    # if the date changes then the mean is calculated + a new csv is created for minutes data 
    if df['date'][ind].date() > date:
        # saving mean data for readings between 23:00 and 00:00
        end_time = df['date'][ind] # datetime.no()
        start_time = end_time - timedelta(hours=1)
        selected_rows = out_miute[(out_miute['date'] >= start_time) & (out_miute['date'] <= end_time)]
        mean = selected_rows.mean()
        # saves means to out_hour
        out_hour.loc[len(out_hour.index)] = [df['date'][ind], df['date'][ind].hour, mean['temp'], mean['humidity'], mean['pressure'], mean['altitude']]
        
        # generating names for outputs
        minutes_file = date.strftime("%Y%m%d") + "_minutes.csv" # datetime.now().strftime("%Y%m%d") 
        hour_file = date.strftime("%Y%m%d") + "_hours.csv" # datetime.now().strftime("%Y%m%d")  
        # saving data 
        out_miute.to_csv(os.path.join(out_path, minutes_file), index=False)
        out_hour.to_csv(os.path.join(out_path, hour_file), index=False)
        # clear dataframes
        out_miute = pd.DataFrame(columns = ['date','temp', 'humidity', 'pressure', 'altitude'])
        out_hour = pd.DataFrame(columns = ['date', 'hour', 'temp', 'humidity', 'pressure', 'altitude'])
        

    minute = df['date'][ind].minute
    hour = df['date'][ind].hour
    date = df['date'][ind].date()

(3395, 5)


C:\Users\Ukasz\Anaconda3\envs\geopandas\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
C:\Users\Ukasz\Anaconda3\envs\geopandas\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
C:\Users\Ukasz\Anaconda3\envs\geopandas\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


### API

In [15]:
'''

'''

import json
import requests
import pprint


def get_data_weather_api():
    '''
    free api up to 10 000 requests per day
    '''
    url = f"https://api.open-meteo.com/v1/forecast?latitude=53.45&longitude=19.95&current=temperature_2m"
    response = requests.get(url)
    response = response.json()
    
    return response
    
    
outside_temp = get_data_weather_api()
outside_temp = outside_temp['current']['temperature_2m'] # gives float
outside_temp

2.8

In [14]:
data['current']['temperature_2m']

float

In [18]:
def api():
    '''
    free api up to 10 000 requests per day
    '''
    url = f"https://api.open-meteo.com/v1/forecast?latitude=53.45&longitude=19.95&current=temperature_2m,,relative_humidity_2m,wind_speed_10m"
    response = requests.get(url)
    response = response.json()
    pprint.pprint(response) # data is in UTC
    return response
    
    
api_data = get_data_weather_api()
outside_temp = api_data['current']['temperature_2m'] #°C 
outside_humidity = api_data['current']['relative_humidity_2m'] # %
winnd_speed = api_data['current']['wind_speed_10m'] # km/h
api_data

{'current': {'interval': 900,
             'relative_humidity_2m': 82,
             'temperature_2m': 2.8,
             'time': '2024-03-04T21:15',
             'wind_speed_10m': 13.7},
 'current_units': {'interval': 'seconds',
                   'relative_humidity_2m': '%',
                   'temperature_2m': '°C',
                   'time': 'iso8601',
                   'wind_speed_10m': 'km/h'},
 'elevation': 195.0,
 'generationtime_ms': 0.041961669921875,
 'latitude': 53.4375,
 'longitude': 19.9375,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'utc_offset_seconds': 0}


{'latitude': 53.4375,
 'longitude': 19.9375,
 'generationtime_ms': 0.041961669921875,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 195.0,
 'current_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature_2m': '°C',
  'relative_humidity_2m': '%',
  'wind_speed_10m': 'km/h'},
 'current': {'time': '2024-03-04T21:15',
  'interval': 900,
  'temperature_2m': 2.8,
  'relative_humidity_2m': 82,
  'wind_speed_10m': 13.7}}

In [21]:
winnd_speed

13.7

In [22]:
from datetime import datetime, timedelta

end_time = datetime.now() # datetime.no()
start_time = end_time - timedelta(minutes=5)


In [23]:
print(start_time, end_time)

2024-03-04 22:50:11.520224 2024-03-04 22:55:11.520224
